# Exploring GPT

## Using the API

Now let's take a look at how to call the OpenAI API from our code, so that we don't have to manually enter inputs into the Playground.  

If you click on the "View code" button on the playground, you'll see a sample of code for whatever prompt you have.  For example, here's the code that we have for our few-shot learning that generates a subject to study for a topic that someone is interested in:

```python
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="text-davinci-002",
  prompt="how to program in Python - computer science\nfactors leading up to WW2 - history\nbranches of government - political science\nShakespeare's plays - English\ncellular respiration - biology\nrespiratory disease - medical\nhow to sculpt - art",
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
```
This is python code, so it'll be pretty easy for us to use this as a starting point and to modify it to create a function that we can call.


First, you'll need install the OpenAPI via pip.  You can use Unix commands in a colab notebook by prefixing them with an exclamation point. The example below uses a `%` prefix instead—this guarantees that the installation is done using the same Python version that your notebook is using. Change line 2 from `%pip...` to `!pip...` if you're running into trouble. (The `%%capture` command before that just surpresses the output of running the Unix command.  You can remove it if you want to see the progress of the command).


In [ ]:
%%capture
%pip install openai

Next, you will enter your secret key for the OpenAI API. You can generate your OpenAI API key [here](https://platform.openai.com/api-keys).  

We will enter it as a password, so that the raw text of it doesn't get saved in your Python notebook. That would be bad in the case that you upload or share your notebook, because then other people could use your key and have you pay for their usage.

In [3]:
from getpass import getpass
import openai
import os

print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key

Enter OpenAI API key:


Now let's write a function that takes a topic as input and then outputs a subject to study if you want to learn about that topic.

In [4]:
import openai
import os
import time

def generate_subject_few_shot(topic):
  few_shot_prompt = """how to program in Python - computer science
factors leading up to WW2 - history
branches of government - political science
Shakespeare's plays - English
cellular respiration - biology
respiratory disease - medical
how to sculpt - art
"""

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "You should recognize a pattern in the prompts and generate a completion based on that pattern."}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : few_shot_prompt + topic + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  # I recommend putting a short wait after each call,
  # since the rate limit for the platform is 60 requests/min.
  # (This increases to 3000 requests/min after you've been using the platform for 2 days).
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  return response.choices[0].message.content.strip()

topic = "cellular respiration"
generate_subject_few_shot(topic)

'biology'

That's it!  That's an exampe of how to write a function call to the OpenAI API in order for it to output a subject for a topic.

Here is some information about the different arguments that we to the `openai.Completion.create` call:
 * `model` – OpenAI offers many different versions of their chat model. We'll use GPT3.5-turbo, which is functional without being [the most expensive to run](https://openai.com/api/pricing/).
 * `messages` - this is the set of messages that will instruct how the model should respond. The first message is a system prompt—an overall setting that specifies how the model should behave. In this case, the second message is the "training set" of examples followed by the last topic that we want to generate a subject for.
 * `temperature` - controls how much of the probability distribution the model will use when it is generating each token. 1.0 means that it samples from the complete probability distrubiton, 0.7 means that it drops the bottom 30% of the least likely tokens when it is sampling. 0.0 means that it will perform deterministically and always output the single most probable token for each context.
 * `top_p` - is an alternative way of controling the sampling.
 * `frequency_penalty` and `presence_penalty` are two ways of reduing the model from repeating the same words in one output.  You can set these to be >0 if you're seeing a lot of repetition in your output.
 * `max_tokens` is the maximum length in tokens that will be output by calling the function.  A token is a subword unit.  There are roughly 2 or 3 tokens per word on average.
 * `stop` is a list of stop sequences.  The model will stop generating output once it generates one of these strings, even if it hasn't reached the max token length. By default this is set to a special token `<|endoftext|>`.

You can read more about [the Chat Completion API call in the documentation](https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models).

# Zero shot learning

In addition to few shot learning, GPT can sometimes also perform "zero shot learning" where instead of giving it several examples of what we want it to do, we can instead give it instructions of what we want it to do.

For example, for our topic - subject task we could give GPT the prompt

> Given a topic, output the subject that a student should study if they want to know more about that topic.

Then if we append
> cellular respiration -

GPT will output `biology`—or something similar. Since there is no training, you might have to accept slight variability in the output. `biology` and `biochemistry` are both reasonable outputs for the input `cellular respiration -`.

Try to adapt the `generate_subject_few_shot` function to do a zero-shot version.

In [7]:
def generate_subject_zero_shot(topic):
  # TODO - write this function
  zero_shot_prompt = """Given a topic, output the subject that a student should study if they want to know more about that topic.
"""

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Give a single subject"}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + topic + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  # I recommend putting a short wait after each call,
  # since the rate limit for the platform is 60 requests/min.
  # (This increases to 3000 requests/min after you've been using the platform for 2 days).
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  return response.choices[0].message.content.strip()

topic = "cellular respiration"
generate_subject_zero_shot(topic)

'Biology'

A very cool recent finding is that training procedure for LLMs can be changed to improve this instruction following behavior.  If LLMs are [trained to do multiple tasks through prompting](https://arxiv.org/abs/2110.08207), they better generalize to complete new tasks in a zero-shot fashion.

Try writing zero-shot prompts to do the following tasks:
1. Summarize a Wikipedia article.
2. Answer questions about an article.
3. Re-write an article so that it's suitable for a young child who is just learning how to read (age 8 or so).
4. Translate an article from Russian into English.

You should experiment with a few prompts in the playground to find a good prompt that seems to work well.

In [23]:
def summarize(article_text):
  # TODO - write this function
  zero_shot_prompt = """Given an article, output a short summary of it in enlgish.
                      """

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Focus on the subject of the article"}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + article_text + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  summary = response.choices[0].message.content.strip()
  return summary

def answer_question(article_text, question):
  # TODO - write this function
  zero_shot_prompt = """Given an article, answer the question asked.
                      """

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Answer the question asked"}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + article_text + question + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  summary = response.choices[0].message.content.strip()
  return summary
  answer = ""
  return answer

def simplify(article_text):
  # TODO - write a function to re-write an article so that it's suitable for a young child.
  zero_shot_prompt = """Given an article, output a short summary of it in enlgish at a child's reading level.
                      """

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Make sure the summary is at a child's reading level"}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + article_text + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  summary = response.choices[0].message.content.strip()
  return summary

def translate(article_text, source_language, target_language):
  # TODO - write a function to translate an article from a source language to a target language.
  zero_shot_prompt = "Translate an article from " + source_language + " to " + target_language + article_text + "."

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Follow the prompt"}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + article_text + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  summary = response.choices[0].message.content.strip()
  return summary


Show your outputs in your prompts.  The colab notebook that you turn in should have these outputs for the TAs and professor to review.

In [17]:
article_text = """
Formula One, commonly known as Formula 1 or F1, is the highest class of international racing for open-wheel single-seater formula racing cars sanctioned by the Fédération Internationale de l'Automobile (FIA). The FIA Formula One World Championship has been one of the world's premier forms of motorsport since its inaugural running in 1950. The word formula in the name refers to the set of rules all participants' cars must follow. A Formula One season consists of a series of races, known as Grands Prix. Grands Prix take place in multiple countries and continents on either purpose-built circuits or closed roads.
A point-system is used at Grands Prix to determine two annual World Championships: one for the drivers, and one for the constructors (the teams). Each driver must hold a valid Super Licence, the highest class of racing licence the FIA issues, and the races must be held on grade one tracks, the highest grade rating the FIA issues for tracks.
Formula One cars are the world's fastest regulated road-course racing cars, owing to high cornering speeds achieved by generating large amounts of aerodynamic downforce, much of which is generated by front and rear wings. The cars depend on electronics, aerodynamics, suspension, and tyres. Traction control, launch control, and automatic shifting, and other electronic driving aids were first banned in 1994. They were briefly reintroduced in 2001, and have more recently been banned since 2004 and 2008, respectively.[1]
With the average annual cost of running a team—designing, building, and maintaining cars, pay, transport—at approximately £220,000,000 (or $265,000,000),[2] Formula One's financial and political battles are widely reported. The Formula One Group is owned by Liberty Media, which acquired it in 2017 from private-equity firm CVC Capital Partners for £6.4 billion ($8 billion).[3][4]
Formula One originated from the World Manufacturers' Championship (1925–1930) and European Drivers' Championship (1931–1939). The formula is a set of rules that all participants' cars must follow. Formula One was a formula agreed upon in 1946 to officially become effective in 1947. The first Grand Prix in accordance with the new regulations was the 1946 Turin Grand Prix, anticipating the formula's official start.[5][6] Before World War II, a number of Grand Prix racing organisations made suggestions for a new championship to replace the European Championship, but due to the suspension of racing during the conflict, the new International Formula for cars did not become formalised until 1946, to become effective in 1947. The new World Championship was instituted to commence in 1950.[7]
The first world championship race, the 1950 British Grand Prix, took place at Silverstone Circuit in the United Kingdom on 13 May 1950.[8] Giuseppe Farina, competing for Alfa Romeo, won the first Drivers' World Championship, narrowly defeating his teammate Juan Manuel Fangio. Fangio won the championship in 1951, 1954, 1955, 1956, and 1957.[9] This set the record for the most World Championships won by a single driver, a record that stood for 46 years until Michael Schumacher won his sixth championship in 2003.[9]
Juan Manuel Fangio's 1951 title-winning Alfa Romeo 159
A Constructors' Championship was added in the 1958 season. Stirling Moss, despite being regarded as one of the greatest Formula One drivers in the 1950s and 1960s, never won the Formula One championship.[10] Between 1955 and 1961, Moss finished second in the championship four times and third the other three times.[11][12] Fangio won 24 of the 52 races he entered—still the record for the highest Formula One winning percentage by an individual driver.[13] National championships existed in South Africa and the UK in the 1960s and 1970s. Promoters held non-championship Formula One events for many years. Due to the increasing cost of competition, the last of these was held in 1983.[14][15]
This era featured teams managed by road-car manufacturers, such as Alfa Romeo, Ferrari, Mercedes-Benz and Maserati. The first seasons featured prewar cars like Alfa Romeo's 158, which were front-engined, with narrow tyres and 1.5-litre supercharged or 4.5-litre naturally aspirated engines. The 1952 and 1953 seasons were run to Formula Two regulations, for smaller, less powerful cars, due to concerns over the dearth of Formula One cars.[16][17] When a new Formula One formula for engines limited to 2.5 litres was reinstated for the 1954 world championship, Mercedes-Benz introduced its W196, which featured things never seen on Formula One cars before, such as desmodromic valves, fuel injection, and enclosed streamlined bodywork. Mercedes drivers won the championship for the next two years, before the team withdrew from all motorsport competitions due to the 1955 Le Mans disaster.[18][19]
"""

summarize(article_text)

'The article provides an overview of Formula One (F1), the highest class of international racing for open-wheel single-seater formula cars. It discusses the history of Formula One, the regulations, the championships for drivers and constructors, as well as the financial and political aspects of the sport. The article also mentions the evolution of Formula One cars, the introduction of various technological advancements, and notable drivers such as Juan Manuel Fangio and Stirling Moss.'

In [20]:
article_text = """
Formula One, commonly known as Formula 1 or F1, is the highest class of international racing for open-wheel single-seater formula racing cars sanctioned by the Fédération Internationale de l'Automobile (FIA). The FIA Formula One World Championship has been one of the world's premier forms of motorsport since its inaugural running in 1950. The word formula in the name refers to the set of rules all participants' cars must follow. A Formula One season consists of a series of races, known as Grands Prix. Grands Prix take place in multiple countries and continents on either purpose-built circuits or closed roads.
A point-system is used at Grands Prix to determine two annual World Championships: one for the drivers, and one for the constructors (the teams). Each driver must hold a valid Super Licence, the highest class of racing licence the FIA issues, and the races must be held on grade one tracks, the highest grade rating the FIA issues for tracks.
Formula One cars are the world's fastest regulated road-course racing cars, owing to high cornering speeds achieved by generating large amounts of aerodynamic downforce, much of which is generated by front and rear wings. The cars depend on electronics, aerodynamics, suspension, and tyres. Traction control, launch control, and automatic shifting, and other electronic driving aids were first banned in 1994. They were briefly reintroduced in 2001, and have more recently been banned since 2004 and 2008, respectively.[1]
With the average annual cost of running a team—designing, building, and maintaining cars, pay, transport—at approximately £220,000,000 (or $265,000,000),[2] Formula One's financial and political battles are widely reported. The Formula One Group is owned by Liberty Media, which acquired it in 2017 from private-equity firm CVC Capital Partners for £6.4 billion ($8 billion).[3][4]
Formula One originated from the World Manufacturers' Championship (1925–1930) and European Drivers' Championship (1931–1939). The formula is a set of rules that all participants' cars must follow. Formula One was a formula agreed upon in 1946 to officially become effective in 1947. The first Grand Prix in accordance with the new regulations was the 1946 Turin Grand Prix, anticipating the formula's official start.[5][6] Before World War II, a number of Grand Prix racing organisations made suggestions for a new championship to replace the European Championship, but due to the suspension of racing during the conflict, the new International Formula for cars did not become formalised until 1946, to become effective in 1947. The new World Championship was instituted to commence in 1950.[7]
The first world championship race, the 1950 British Grand Prix, took place at Silverstone Circuit in the United Kingdom on 13 May 1950.[8] Giuseppe Farina, competing for Alfa Romeo, won the first Drivers' World Championship, narrowly defeating his teammate Juan Manuel Fangio. Fangio won the championship in 1951, 1954, 1955, 1956, and 1957.[9] This set the record for the most World Championships won by a single driver, a record that stood for 46 years until Michael Schumacher won his sixth championship in 2003.[9]
Juan Manuel Fangio's 1951 title-winning Alfa Romeo 159
A Constructors' Championship was added in the 1958 season. Stirling Moss, despite being regarded as one of the greatest Formula One drivers in the 1950s and 1960s, never won the Formula One championship.[10] Between 1955 and 1961, Moss finished second in the championship four times and third the other three times.[11][12] Fangio won 24 of the 52 races he entered—still the record for the highest Formula One winning percentage by an individual driver.[13] National championships existed in South Africa and the UK in the 1960s and 1970s. Promoters held non-championship Formula One events for many years. Due to the increasing cost of competition, the last of these was held in 1983.[14][15]
This era featured teams managed by road-car manufacturers, such as Alfa Romeo, Ferrari, Mercedes-Benz and Maserati. The first seasons featured prewar cars like Alfa Romeo's 158, which were front-engined, with narrow tyres and 1.5-litre supercharged or 4.5-litre naturally aspirated engines. The 1952 and 1953 seasons were run to Formula Two regulations, for smaller, less powerful cars, due to concerns over the dearth of Formula One cars.[16][17] When a new Formula One formula for engines limited to 2.5 litres was reinstated for the 1954 world championship, Mercedes-Benz introduced its W196, which featured things never seen on Formula One cars before, such as desmodromic valves, fuel injection, and enclosed streamlined bodywork. Mercedes drivers won the championship for the next two years, before the team withdrew from all motorsport competitions due to the 1955 Le Mans disaster.[18][19]
"""
questions = [
    "Who owns formula one?",
    "What is the average annual cost of running a Formula One team?",
    "What year was formula one first started?",
    "Where did the first race take place?",
    "What is a current team on the grid today?",
]

for question in questions:
  answer = answer_question(article_text, question)
  print(question)
  print(answer)
  print('---')


Who owns formula one?
Formula One is owned by Liberty Media, which acquired it in 2017 from private-equity firm CVC Capital Partners for £6.4 billion ($8 billion).
---
What is the average annual cost of running a Formula One team?
The average annual cost of running a Formula One team is approximately £220,000,000 or $265,000,000.
---
What year was formula one first started?
Formula One was first started in 1950.
---
Where did the first race take place?
The first race took place at the 1950 British Grand Prix, held at the Silverstone Circuit in the United Kingdom on 13 May 1950.
---
What is a current team on the grid today?
One of the current teams on the Formula One grid today is Mercedes-AMG Petronas Formula One Team.
---


In [22]:
article_text = """
Formula One, commonly known as Formula 1 or F1, is the highest class of international racing for open-wheel single-seater formula racing cars sanctioned by the Fédération Internationale de l'Automobile (FIA). The FIA Formula One World Championship has been one of the world's premier forms of motorsport since its inaugural running in 1950. The word formula in the name refers to the set of rules all participants' cars must follow. A Formula One season consists of a series of races, known as Grands Prix. Grands Prix take place in multiple countries and continents on either purpose-built circuits or closed roads.
A point-system is used at Grands Prix to determine two annual World Championships: one for the drivers, and one for the constructors (the teams). Each driver must hold a valid Super Licence, the highest class of racing licence the FIA issues, and the races must be held on grade one tracks, the highest grade rating the FIA issues for tracks.
Formula One cars are the world's fastest regulated road-course racing cars, owing to high cornering speeds achieved by generating large amounts of aerodynamic downforce, much of which is generated by front and rear wings. The cars depend on electronics, aerodynamics, suspension, and tyres. Traction control, launch control, and automatic shifting, and other electronic driving aids were first banned in 1994. They were briefly reintroduced in 2001, and have more recently been banned since 2004 and 2008, respectively.[1]
With the average annual cost of running a team—designing, building, and maintaining cars, pay, transport—at approximately £220,000,000 (or $265,000,000),[2] Formula One's financial and political battles are widely reported. The Formula One Group is owned by Liberty Media, which acquired it in 2017 from private-equity firm CVC Capital Partners for £6.4 billion ($8 billion).[3][4]
Formula One originated from the World Manufacturers' Championship (1925–1930) and European Drivers' Championship (1931–1939). The formula is a set of rules that all participants' cars must follow. Formula One was a formula agreed upon in 1946 to officially become effective in 1947. The first Grand Prix in accordance with the new regulations was the 1946 Turin Grand Prix, anticipating the formula's official start.[5][6] Before World War II, a number of Grand Prix racing organisations made suggestions for a new championship to replace the European Championship, but due to the suspension of racing during the conflict, the new International Formula for cars did not become formalised until 1946, to become effective in 1947. The new World Championship was instituted to commence in 1950.[7]
The first world championship race, the 1950 British Grand Prix, took place at Silverstone Circuit in the United Kingdom on 13 May 1950.[8] Giuseppe Farina, competing for Alfa Romeo, won the first Drivers' World Championship, narrowly defeating his teammate Juan Manuel Fangio. Fangio won the championship in 1951, 1954, 1955, 1956, and 1957.[9] This set the record for the most World Championships won by a single driver, a record that stood for 46 years until Michael Schumacher won his sixth championship in 2003.[9]
Juan Manuel Fangio's 1951 title-winning Alfa Romeo 159
A Constructors' Championship was added in the 1958 season. Stirling Moss, despite being regarded as one of the greatest Formula One drivers in the 1950s and 1960s, never won the Formula One championship.[10] Between 1955 and 1961, Moss finished second in the championship four times and third the other three times.[11][12] Fangio won 24 of the 52 races he entered—still the record for the highest Formula One winning percentage by an individual driver.[13] National championships existed in South Africa and the UK in the 1960s and 1970s. Promoters held non-championship Formula One events for many years. Due to the increasing cost of competition, the last of these was held in 1983.[14][15]
This era featured teams managed by road-car manufacturers, such as Alfa Romeo, Ferrari, Mercedes-Benz and Maserati. The first seasons featured prewar cars like Alfa Romeo's 158, which were front-engined, with narrow tyres and 1.5-litre supercharged or 4.5-litre naturally aspirated engines. The 1952 and 1953 seasons were run to Formula Two regulations, for smaller, less powerful cars, due to concerns over the dearth of Formula One cars.[16][17] When a new Formula One formula for engines limited to 2.5 litres was reinstated for the 1954 world championship, Mercedes-Benz introduced its W196, which featured things never seen on Formula One cars before, such as desmodromic valves, fuel injection, and enclosed streamlined bodywork. Mercedes drivers won the championship for the next two years, before the team withdrew from all motorsport competitions due to the 1955 Le Mans disaster.[18][19]
"""

simplify(article_text)

'Formula One, also known as F1, is a really fast and exciting racing sport. Drivers race in special cars on cool tracks all around the world. They have to follow certain rules called "formula." Each year, the drivers and the teams that make the cars compete to win championships. The cars are super fast and use special technology to go even faster. The sport has been around for a long time and has a lot of history. The races are fun to watch and the drivers are super talented.'

In [24]:
russian_article = """
Избранный президент США Дональд Трамп заявил, что может ввести стопроцентные пошлины против стран БРИКС, если они не откажутся от планов по созданию единой валюты объединения в качестве альтернативы доллару.
"Мы требуем от этих стран обязательства не создавать новую валюту БРИКС", - написал Трамп в социальной сети Truth Social.
Он также потребовал, что бы страны БРИКС не поддерживали никакую другую валюту, кроме "могущественного доллара США".
"Иначе они столкнутся с 100% пошлинами", - пригрозил Трамп, добавив, что для этих стран может быть ограничена торговля с США.
"""

source_language = "Russian"
target_language = "English"
translate(russian_article, source_language, target_language)

'The elected President of the United States, Donald Trump, has stated that he may impose 100% tariffs against BRICS countries if they do not abandon plans to create a unified currency as an alternative to the dollar.'

## TODO - Pick your own task

For this section you should pick some task that you'd like to have GPT do.  Add a description and code to your notebook here.  You should:
1. Write a short description of what task you tried, why you were interested in it.
2. Give some code so that we can reproduce what you did via an Open API call.  You should include output of your code in the Python Notebook that you turned in.
3. Write a short qualitative analysis of whether or not GPT did the task well.

TODO - your task description

In [31]:
# TODO your code
def top_five(sport):
  zero_shot_prompt = "Give the top 5 all time players in the professional sport of " + sport + "."
  system_message = {"role" : "system", "content" : "Output this in a single sentence. Adhere to american sport labels."}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + sport + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  
  time.sleep(1)

  
  summary = response.choices[0].message.content.strip()
  return summary

sport = "football"
top_five(sport)

'The top 5 all-time players in professional football are Tom Brady, Jerry Rice, Lawrence Taylor, Jim Brown, and Reggie White.'

TODO - write a short paragraph giving your qualitative analysis of how well GPT did for your task.

In [ ]:
# This is the short paragraph explaining my code
"""This goal of this is to input a sport that exists in the world and have the top 5 players outputed. This is a slightly subjective subject, so the results will vary from case to case.
The function outputs are acceptable in that they stay consistent and when given a sport that has alternative names, it adheres to the standard of the US-labeling system. This is instructed
to output the answer in a single sentence, but it can be altered to output it line by line instead, giving a more list like aesthetic."""

# Fine Tuning

In addition to zero-shot and few-shot learning, another way of getting large language models to do your tasks is via a process called "fine tuning".  In fine-tuning the model updates its parameters so that it performs well on many training examples.  The training examples are in the form of input prompts paired with gold standard completions.

Large language models are pre-trained to perform well on general tasks like text completion but not on the specific task that you might be interested in.  The models can be fine tuned to perform you task, starting with the model parameters that are good for the general setting, and then updating them to be good for your task.

We'll walk through how to fine-tune GPT for a task.


For this example, we will show you how to fine tune GPT to write biographies. From data in the info boxes in Wikipedia pages.  For instance, given this input

```
notable_type: scientist
name: Zulima Aban
gender: female
birth_date: 05 December 1905
birth_place: Valencia, Spain
death_date: 09 August 1983
death_place: Detroit, Michigan, U.S.
death_cause: Pulmonary embolism
occupation: Astronomer
fields: Astrophysics, Computer Science, Computer Graphics, Interface Design, Image Synthesis
known_for: The Search for Planet Nine
hometown: Detroit, Michigan, U.S.
nationality: Venezuelan
citizenship: Spanish, American
alma_mater: University of Valencia (B.Sc.), University of Madrid (Ph.D.)
thesis_title: The Formation of Planets by the Accretion of Small Particles
thesis_year: 1956
doctoral_advisor: Angela Carter
awards: Spanish Academy of Science, Spanish Academy of Engineering, German Aerospace Prize, IEEE Medal of Honor, IEEE John von Neumann Medal, IEEE Jack S. Kilby Signal Processing Medal, United Nations Space Pioneer Award, Wolf Prize in Physics
institutions: Oberlin College, University of Valencia, Instituto de Astrofísica de Andalucía (CSIC), University of Southern California, Space Telescope Science Institute (STScI)
notable_students: Ryan Walls
influences: Immanuel Kant, Albert Einstein, Kurt Gödel, Gottfried Leibniz, Richard Feynman, Werner Heisenberg, William Kingdon Clifford, Sir Arthur Eddington
influenced: Joseph Weinberg
mother: Ana Aban
father: Joaquín Aban
partner: Georgina Abbott
children: Robert, Peter, Sarah
```

The fine-tuned model will generate this output:

> Zulima Aban was a Venezuelan astronomer, who was born on 05 December 1905 in Valencia, Spain to Ana Aban and Joaquín Aban. Her career involved the fields of Astrophysics, Computer Science, Computer Graphics, Interface Design, Image Synthesis. Aban was known for The Search for Planet Nine. Aban went to University of Valencia (B.Sc.), University of Madrid (Ph.D.). Aban's thesis title was The Formation of Planets by the Accretion of Small Particles in 1956. Her doctoral advisor was Angela Carter. Aban received Spanish Academy of Science, Spanish Academy of Engineering, German Aerospace Prize, IEEE Medal of Honor, IEEE John von Neumann Medal, IEEE Jack S. Kilby Signal Processing Medal, United Nations Space Pioneer Award, Wolf Prize in Physics. Aban went to Oberlin College, University of Valencia, Instituto de Astrofísica de Andalucía (CSIC), University of Southern California, Space Telescope Science Institute (STScI). Her notable students were Ryan Walls. Aban was influenced by Immanuel Kant, Albert Einstein, Kurt Gödel, Gottfried Leibniz, Richard Feynman, Werner Heisenberg, William Kingdon Clifford, Sir Arthur Eddington and she infuenced Joseph Weinberg. Aban was married to Georgina Abbott and together had three children, Robert, Peter, Sarah. Aban died on 09 August 1983 in Detroit, Michigan, U.S due to Pulmonary embolism.

The dataset that we will use was created for the paper [SynthBio: A Case Study in Human-AI Collaborative Curation of Text Datasets](https://www.cis.upenn.edu/~ccb/publications/synthbio.pdf) by Ann Yuan, Daphne Ippolito, Vitaly Nikolaev, Chris Callison-Burch, Andy Coenen, and Sebastian Gehrmann. It was published in NeurIPS 2021.  The goal of the paper was to create a curated dataset for training large language models on synthetic data with the goal of avoiding the gender and geographic bias that is naturally present in Wikipedia due to cultural and historic reasons.


## Load the data

In [72]:
# !wget https://raw.githubusercontent.com/artificial-intelligence-class/artificial-intelligence-class.github.io/master/homeworks/large-LMs/SynthBio_train.json

# If you're running this locally on a Mac, you can use the following command instead.
!curl -o SynthBio_train.json https://raw.githubusercontent.com/artificial-intelligence-class/artificial-intelligence-class.github.io/master/homeworks/large-LMs/SynthBio_train.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5671k  100 5671k    0     0  6228k      0 --:--:-- --:--:-- --:--:-- 6225k


In [73]:
# Load a file called 'SynthBio.json' which is a list of json objects.

import json
import random

def load_wiki_bio_data(filename='SynthBio_train.json', num_bios=100, randomized=True):
  with open(filename) as f:
    synth_bio_data = json.load(f)
  random.shuffle(synth_bio_data)
  bios = []
  for data in synth_bio_data:
    notable_type = data['notable_type']
    attributes = "notable_type: {notable_type} | {other_attributes}".format(
        notable_type = notable_type,
        other_attributes = data['serialized_attrs']
    )
    biography = data['biographies'][0]
    bios.append((attributes.replace(" | ", "\n"), biography))
  return bios[:min(num_bios, len(bios))]

wiki_bios = load_wiki_bio_data()
attributes, bio = wiki_bios[0]
print(attributes)
print('---')
bio


notable_type: theologian
name: Ibrahim Al-Ayoub
gender: female
nationality: Jordanian
birth_date: 27 July 1898
birth_place: Tafila, Jordan
death_date: 1 March, 1989
death_place: Amman, Jordan
death_cause: stroke
alma_mater: unknown
occupation: writer, philosopher
notable_works: A Prophet's Cry, Wisdom for Life, Life and Living, The Tree with a Thousand Souls
main_interests: Eastern Orthodoxy, philosophy, self-transcendence
mother: Al-Ayoubi
father: Al-Ayoubi
partner: Issa Al-Boustani
children: none
---


'Ibrahim Al-Ayoub (born 27 July 1898 in Tafila, Jordan - died 1 March 1989 in Amman, Jordan) was a Jordanian writer, philosopher. She was best known for his book "A Prophet\'s Cry", which detailed his thoughts on the life and suffering of Jesus. She died in Amman, Jordan. She is the daughter of Al-Ayoubi and Al-Ayoubi.'

## Format Data for Fine-Tuning

Below, I show how to format data to fine-tune OpenAI.  The OpenAI API documentation has a [guide to fine-tuning models](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset) that you should read.   The basic format of fine-tuning data is a JSONL file (one JSON object per line) with an entry for `"messages"` that itself contains a list of `message` objects. Each `message` object consists of an entry for a `role` and `content`.

```
{"messages": [{"role": "system", "content": "Explanation for how LLM should behave."}, {"role": "user", "content": "User's prompt 1"}, {"role": "assistant", "content": "Expected response 1."}]}
{"messages": [{"role": "system", "content": "Explanation for how LLM should behave."}, {"role": "user", "content": "User's prompt 2"}, {"role": "assistant", "content": "Expected response 2."}]}
{"messages": [{"role": "system", "content": "Explanation for how LLM should behave."}, {"role": "user", "content": "User's prompt 3"}, {"role": "assistant", "content": "Expected response 3."}]}
...
```

In the code below, I'll extract a prompt that contains the `attributes` variable from the intent determination data, and I'll have the completion be the `biography` variable.

In [111]:
import json

def create_wikibio_finetuning_data(wikibios, fine_tuning_filename):
  fine_tuning_data = []
  system_message = {"role" : "system", "content" : "You should generate a biography for a person based on the details provided."}

  for attributes, bio in wikibios:
    prompt = "{attributes}\n---\n".format(attributes=attributes)
    completion = "Biography: {bio}\n###".format(bio=bio)
    data = {"messages" : [system_message, {"role" : "user", "content" : prompt}, {"role" : "assistant", "content" : completion}]}
    fine_tuning_data.append(data)

  random.shuffle(fine_tuning_data)
  with open(fine_tuning_filename, 'w') as out:
    for data in fine_tuning_data:
        out.write(json.dumps(data))
        out.write('\n')


fine_tuning_filename='wikibio_finetuning_data.jsonl'
create_wikibio_finetuning_data(wiki_bios, fine_tuning_filename)

Next, we'll perform fine-tuning with this data using OpenAI.

In [112]:
%%capture
%pip install --upgrade openai
%pip install jsonlines
%pip install wandb

Once you've got access to the OpenAI API, you can find your OpenAI API key [here](https://beta.openai.com/account/api-keys).

In [39]:
import os
import openai

from getpass import getpass
print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key

Enter OpenAI API key:


In [113]:
!head '{fine_tuning_filename}'

{"messages": [{"role": "system", "content": "You should generate a biography for a person based on the details provided."}, {"role": "user", "content": "notable_type: musician\nname: Simon Nda'Okongo\nbirth_name: Simon Nda'Okongo\nalias: B'June\ngender: non-binary\nbirth_date: 27 August 1916\nbirth_place: Libreville, Gabon\ndeath_date: 1 January 1982\ndeath_place: Libreville, Gabon\ndeath_cause: complications due to pneumonia\nresting_place: Mortuary of the Saint-Pierre-et-Saint-Paul Cathedral, Libreville\ninstrument: drums\ngenre: folktronica\nhometown: Libreville, Gabon\nnationality: Gabonese\ncitizenship: British and American by descent\neducation: University of California, Los Angeles - School of Arts and Architecture, New York University\nyears_active: 1940-1950\nlabel: Paramount Records\nassociated_acts: John Coltrane, Miles Davis, Billy Strayhorn, Duke Ellington\nmother: Eulalie Enowa Ekongo\nfather: Marcel B. Nda'kongo\npartner: Dorothy Pak\nchildren: Brice and Tr\u00e9sor Nda'

## Run the fine-tuning API

Next, we'll make the fine tuning call using the Python API. First, we must upload the training data to OpenAI. Once we do that, we can use the `id` of the uploaded data to perform the fine-tuning.

In [41]:
client = openai.OpenAI()

client.files.create(
  file=open("wikibio_finetuning_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-BULADKjXbSjKvhVAW6E1rz', bytes=145976, created_at=1732997738, filename='wikibio_finetuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

Make sure to change the string used for your `training_file` to the `id` that was returned as a result of the previous step.

**Do NOT repeatedly execute the following cell.**  It will cost you upwards of a dollar each time you run it. Run it once, and then move on to the next cell, where you can check its progress. The amount of time required for the fine-tune job to finish can be highly variable. Do not—again, **DO NOT**—repeatedly execute the following cell.

In [42]:
client.fine_tuning.jobs.create(
  training_file="file-BULADKjXbSjKvhVAW6E1rz",
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-Vod0juyldT7zqUr8CLngUUZY', created_at=1732997793, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CtG7bJ67coneBV3uvISitdeu', result_files=[], seed=1732171217, status='validating_files', trained_tokens=None, training_file='file-BULADKjXbSjKvhVAW6E1rz', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

You should copy down the fine-tune's `id`.

You can list all of your fine-tune jobs and check their statuses using the following few examples.

In [44]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-Vod0juyldT7zqUr8CLngUUZY")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-Vod0juyldT7zqUr8CLngUUZY", limit=10)


# Cancel a job
# client.fine_tuning.jobs.cancel("my_job_id")

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-klODJjaWwUDkTSjpEtbkyoEv', created_at=1732997974, level='info', message='Step 15/300: training loss=0.92', object='fine_tuning.job.event', data={'step': 15, 'train_loss': 0.9213531613349915, 'total_steps': 300, 'train_mean_token_accuracy': 0.7799999713897705}, type='metrics'), FineTuningJobEvent(id='ftevent-e73bpgpUhjX6U9DLlAyY0FrI', created_at=1732997970, level='info', message='Step 14/300: training loss=1.28', object='fine_tuning.job.event', data={'step': 14, 'train_loss': 1.2765843868255615, 'total_steps': 300, 'train_mean_token_accuracy': 0.7272727489471436}, type='metrics'), FineTuningJobEvent(id='ftevent-HfXexsozjH4OwlhySbbTdOvv', created_at=1732997970, level='info', message='Step 13/300: training loss=0.98', object='fine_tuning.job.event', data={'step': 13, 'train_loss': 0.9794135093688965, 'total_steps': 300, 'train_mean_token_accuracy': 0.8074866533279419}, type='metrics'), FineTuningJobEvent(id='ftevent-Z

You can run your fine tuned model in the OpenAI Playground.  After the model is finished finetuning you'll find it in the Model dropdown menu (you might need to press reload in your browser for your fine-tuned model to appear).

## Call your fine-tuned model from the OpenAI API

Alternately, you can use your fine tuned model via the API by specifying it as the model.  Here's an example:

In [53]:
def generate_bio(attributes, finetuned_model):
  client = openai.OpenAI()
  prompt = "{attributes}\n---\n".format(attributes=attributes)
  system_message = {"role" : "system", "content" : "You should generate a biography for a person based on the details provided."}
  response = client.chat.completions.create(
      model=finetuned_model,
      messages = [
          system_message,
          {"role" : "user", "content" : prompt}
      ],
      temperature=0.7,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["###"]
      )
  return response.choices[0].message.content.strip()

# Replace with your model's name
finetuned_model = "gpt-3.5-turbo-0125"

In [177]:
attributes = """notable_type: computer scienist
alma_mater: Stanford University (BS in Symbolic Systems), University of Edinburgh (PhD in Informatics)
birth_place: California
children: 2
gender: male
main_interests: Artificial Intelligence, Natural Language Processing
name: Chris Callison-Burch
nationality: American
notable_works: Moses: Open source toolkit for statistical machine translation, The Paraphrase Database (PPDB)
occupation: professor
courses_taught: AI, Crowdsourcing and NLP
enrollment_in_most_popular_course: 570 students
institution: University of Pennsylvania"""

biography = generate_bio(attributes, finetuned_model)
print(attributes)
print('---')
biography

notable_type: computer scienist
alma_mater: Stanford University (BS in Symbolic Systems), University of Edinburgh (PhD in Informatics)
birth_place: California
children: 2
gender: male
main_interests: Artificial Intelligence, Natural Language Processing
name: Chris Callison-Burch
nationality: American
notable_works: Moses: Open source toolkit for statistical machine translation, The Paraphrase Database (PPDB)
occupation: professor
courses_taught: AI, Crowdsourcing and NLP
enrollment_in_most_popular_course: 570 students
institution: University of Pennsylvania
---


'Chris Callison-Burch is an American computer scientist known for his work in Artificial Intelligence and Natural Language Processing. He was born in California and holds a Bachelor of Science in Symbolic Systems from Stanford University and a PhD in Informatics from the University of Edinburgh.\n\nChris is a professor at the University of Pennsylvania, where he teaches courses in AI, Crowdsourcing, and NLP. He is a father of two children and has made significant contributions to the field of computer science, particularly through his notable works such as Moses, an open-source toolkit for statistical machine translation, and The Paraphrase Database (PPDB).\n\nWith a passion for advancing the capabilities of AI and NLP, Chris has influenced many students, with his most popular course boasting an enrollment of 570 students. His dedication to research and education has solidified his reputation as a leading figure in the field of computer science.'

## Analyze your model's output

Sometimes the model will add facts that are not present in the attributes.  For instance, one time it said
> He was a member of the research staff at IBM Research in Yorktown Heights.

which is not correct. Another time it said
> His most popular course was on AI, which had 570 students.

which is correct, but not specified in the attirbutes.

Try running your own fine-tuned model until it produces something that wasn't licensed by the attributes.

Save the good runs and the bad run below.

In [ ]:
generations_with_correct_facts = [
    """ Chris Callison-Burch is an American computer scientist known for his work in the fields of Artificial Intelligence and Natural Language Processing. 
        He was born in California and pursued his Bachelor of Science in Symbolic Systems from Stanford University, followed by a PhD in Informatics from the University of Edinburgh.
        \n\nCurrently, Chris Callison-Burch serves as a professor at the University of Pennsylvania, where he teaches courses on AI, Crowdsourcing, and NLP. 
        He has made significant contributions to the field through his notable works such as Moses, an open-source toolkit for statistical machine translation, and The Paraphrase Database (PPDB).
        \n\nChris is a dedicated educator with an enrollment of 570 students in his most popular course. In addition to his academic achievements, he is a father of two children. 
    """,
    """ 
    'Chris Callison-Burch is an American computer scientist known for his work in Artificial Intelligence and Natural Language Processing. 
    He was born in California and went on to earn a BS in Symbolic Systems from Stanford University. 
    He later obtained his PhD in Informatics from the University of Edinburgh.
    \n\nCurrently, Chris serves as a professor at the University of Pennsylvania, where he teaches courses on AI, Crowdsourcing, and NLP. 
    He is a notable figure in the field, with his research focusing on developing innovative tools and technologies. 
    Chris is the creator of Moses, an open-source toolkit for statistical machine translation, and The Paraphrase Database (PPDB).
    \n\nWith a passion for educating the next generation of computer scientists, Chris has taught numerous students, with his most popular course enrolling 570 students. 
    Outside of his academic pursuits, Chris is a father of two children and continues to make significant contributions to the field of computer science.'
    """,
    ]

generation_with_incorrect_facts = """
'Chris Callison-Burch is a renowned computer scientist originally from California, United States. 
He holds a Bachelor of Science degree in Symbolic Systems from Stanford University and a PhD in Informatics from the University of Edinburgh. 
Chris is a dedicated professor at the University of Pennsylvania, where he teaches courses on Artificial Intelligence, Crowdsourcing, and Natural Language Processing.
\n\nHis main research interests lie in Artificial Intelligence and Natural Language Processing, and he has made significant contributions to the field. 
Chris is best known for developing Moses, an open-source toolkit for statistical machine translation, and for creating The Paraphrase Database (PPDB).
\n\nThroughout his career, Chris has inspired and educated numerous students, with his most popular course boasting an enrollment of 570 students. 
In addition to his academic pursuits, Chris is also a loving father of two children and continues to push the boundaries of computer science with his innovative work.'
"""

# Although he does do research, this is not in the attributes
incorrect_facts = [
    "His main research interests lie in Artificial Intelligence and Natural Language Processing, and he has made significant contributions to the field."
    ]

# Fine Tune a New Model

Now that you've seen an example of how to do fine-tuning with the OpenAI API, let's have you write code to fine-tune your own model.

For this model, I'd like you to do the reverse direction of what we just did.  Given a Wikipedia Biograph like this:

> Jill Tracy Jacobs Biden (born June 3, 1951) is an American educator and the current first lady of the United States as the wife of President Joe Biden. She was the second lady of the United States from 2009 to 2017. Since 2009, Biden has been a professor of English at Northern Virginia Community College.

> She has a bachelor's degree in English and a doctoral degree in education from the University of Delaware, as well as master's degrees in education and English from West Chester University and Villanova University. She taught English and reading in high schools for thirteen years and instructed adolescents with emotional disabilities at a psychiatric hospital. From 1993 to 2008, Biden was an English and writing instructor at Delaware Technical & Community College. Biden is thought to be the first wife of a vice president or president to hold a paying job during her husband's tenure.

> Born in Hammonton, New Jersey, she grew up in Willow Grove, Pennsylvania. She married Joe Biden in 1977, becoming stepmother to Beau and Hunter, his two sons from his first marriage. Biden and her husband also have a daughter together, Ashley Biden, born in 1981. She is the founder of the Biden Breast Health Initiative non-profit organization, co-founder of the Book Buddies program, co-founder of the Biden Foundation, is active in Delaware Boots on the Ground, and with Michelle Obama is co-founder of Joining Forces. She has published a memoir and two children's books.

Your model should output something like this:
```
notable_type: First Lady of the United States
name: Jill Biden
gender: female
nationality: American
birth_date: 03 June 1951
birth_place: Hammonton, New Jersey
alma_mater: University of Delaware
occupation: professor of English at Northern Virginia Community College
notable_works: children's books and memoir
main_interests: education, literacy, women's health
partner: Joe Biden
children: Ashley Biden, Beau Biden (stepson), Hunter Biden (stepson)
```


In [442]:
import json

def create_wikibio_parser_finetuning_data(wikibios, fine_tuning_filename):
  # TODO - write your fine-tuning function
  fine_tuning_data = []
  system_message = {"role" : "system", "content" : "Analyse the inputed biography and output the "}

  for attributes, bio in wikibios:
    completion = "{attributes}\n---\n".format(attributes=attributes)
    prompt = "Biography: {bio}\n###".format(bio=bio)
    data = {"messages" : [system_message, {"role" : "user", "content" : prompt}, {"role" : "assistant", "content" : completion}]}
    fine_tuning_data.append(data)

  random.shuffle(fine_tuning_data)
  with open(fine_tuning_filename, 'w') as out:
    for data in fine_tuning_data:
        out.write(json.dumps(data))
        out.write('\n')

fine_tuning_filename='wikibio_parser_finetuning_data.jsonl'
create_wikibio_parser_finetuning_data(wiki_bios, fine_tuning_filename)
!head '{fine_tuning_filename}'

{"messages": [{"role": "system", "content": "Analyse the inputed biography and output the "}, {"role": "user", "content": "Biography: Liza Tugel (20 May 1917 - 25 July 1941) was a Kyrgyzstani horse racer. She was born in Bosteri, Kyrgyzstan. Her active years are from 1925 to 1941. She died of tuberculosis on 25 July 1941 in Kyrgyzstan and was buried in S\u00f6ktsk Gorge. She was born to Yekaterina Sergeyevna Tugel and Kazybek Moldagalloevich Tugel. She attended primary school. Her height was 5ft 1in and weight was 110lb. She married her partner Anastasia Iosifovna Tugel and had one child Leyla.\n###"}, {"role": "assistant", "content": "notable_type: athlete\nname: Liza Tugel\ngender: female\nnationality: Kyrgyzstani\nbirth_date: 20 May 1917\nbirth_place: Bosteri, Kyrgyzstan\ndeath_date: 25 July 1941\ndeath_place: Kyrgyzstan\ndeath_cause: tuberculosis\nresting_place: S\u00f6ktsk Gorge\nsport: horse racing\ncountry: Kyrgyzstan\ncitizenship: none\neducation: primary school\nevent: horse r

In [235]:
# Upload the file to OpenAI's API (only do this when you're confident your data looks right.)
client.files.create(
  file=open("wikibio_parser_finetuning_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-E4frVQmmLqxViko6Vg6QxT', bytes=146405, created_at=1733023431, filename='wikibio_parser_finetuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [236]:
# Create a fine-tune job. Careful! ($$$)
client.fine_tuning.jobs.create(
  training_file="file-E4frVQmmLqxViko6Vg6QxT",
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-LZKJNhsEytVJrSXJ2OOvlpew', created_at=1733023451, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CtG7bJ67coneBV3uvISitdeu', result_files=[], seed=68150432, status='validating_files', trained_tokens=None, training_file='file-E4frVQmmLqxViko6Vg6QxT', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
def parse_bio(biography, finetuned_bio_parser_model):
  # TODO call the API with your fine-tuned model, return a string representing the attributes
  client = openai.OpenAI()
  prompt = "Biography: {biography}\n###".format(biography=biography)
  system_message = {"role" : "system", "content" : "In one string, produce thirty common attributes for the person with the information given."}
  response = client.chat.completions.create(
      model=finetuned_bio_parser_model,
      messages = [
          system_message,
          {"role" : "user", "content" : prompt}
      ],
      temperature=0.7,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["###"]
      )
  return response.choices[0].message.content.strip()


finetuned_bio_parser_model="ft:gpt-3.5-turbo-0125:personal::AZV0fhgm"

In [524]:
# Delete this at the end, this is just to test my tuner 
# bio = "Igor Ivanov was born on October 18, 1970 in Moscow, Russia. His mother is Anastasia Nikolaevna Ivanova, and his father is Nikolay Ivanovich Ivanov. Ivanov is a Russian artist who specializes in oil paintings of portraits, city scenes, still lifes, and flowers. Ivanov attended the Moscow College of Art (now in the Moscow Institute of Painting, Sculpture and Architecture), where he received a bachelor's degree in painting and a master's degree in art history. His notable works include Portraits of Celebrities, Still Life with Flowers, Flowers and Fruit and movement realism. He has received several awards, including the People's Artist of the USSR, Hero of Socialist Labour. Ivanov is also a Professor of the Academy, and he was elected a People's Artist of Russia in 2006. He is married to Alexandra Alexandrovna Vasilieva, and they have two daughters, Alexandra Ivanovna Vasilieva and Anastasia Ivanovna Vasilieva."
bio = "Ansu Ngalaka (10 October 1919 - 31 October 2012) was a Congolese theologian and catholic priest. He was born in Léopoldville, Belgian Congo to a wealthy Christian family. He attended the Louvain Catholic University, Catholic University of Leuven, University of Leuven, Catholic University of Belgium, Catholic University of Paris and his main interest was in Theology of God. He was born to Mwadibu, Agathe Ngalaka and was married to Odette Ngalaka. Ansu was Catholic and he had one child. Anys died due to pancreatic cancer in Paris, France and is buried in Boissy-Saint-Léger."

test_attributes = parse_bio(bio, finetuned_bio_parser_model)
print(test_attributes)

gender: male
nationality: Congolese
birth_date: 10 October 1919
birth_place: Léopoldville, Belgian Congo
death_date: 31 October 2012
death_place: Paris, France
death_cause: pancreatic cancer
resting_place: Boissy-Saint-Léger
alma_mater: Louvain Catholic University, Catholic University of Leuven, University of Leuven, Catholic University of Belgium, Catholic University of Paris
occupation: theologian, catholic priest
notable_works: Theology of God
main_interest: Theology of God
mother: Mwadibu
father: Agathe Ngalaka
partner: Odette Ngalaka
children: one child
---


## Test your parser

Next we will test your parser.  This will involve calling your `parse_bio` function about 250 times, so be sure that you've got it properly debugged and working before running this code.

In [437]:
!curl -o SynthBio_test.json https://raw.githubusercontent.com/artificial-intelligence-class/artificial-intelligence-class.github.io/master/homeworks/large-LMs/SynthBio_test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  649k  100  649k    0     0   805k      0 --:--:-- --:--:-- --:--:--  805k


In [474]:
import json

def load_wiki_bio_test_set(filename='SynthBio_test.json', max_test_items=10, randomized=True):
  """
  Loads our wikibio test set, and returns a list of tuples
  biographies (text), attributes (dictionaires)
  """
  with open(filename) as f:
    synth_bio_data = json.load(f)
  bios = []
  for data in synth_bio_data:
    notable_type = data['notable_type']
    attributes = data['attrs']
    attributes['notable_type'] = notable_type
    biography = data['biographies'][0]
    bios.append((biography, attributes))
  return bios[:min(max_test_items, len(bios))]

def convert_to_dict(predcited_attributes_txt):
  """
  Converts predicted attributes from text format into a dictionary.
  """
  predicted_attributes = {}
  for line in predcited_attributes_txt.split('\n'):
    try:
      attribute, value = line.split(':')
      predicted_attributes[attribute.strip()] = value.strip()
    except ValueError:
      continue
  return predicted_attributes

Helper function for computing precision, recall and f-score.

In [475]:
from collections import Counter

def update_counts(gold_attributes, predicted_attributes, true_positives, false_positives, false_negatives, all_attributes):
  # Compute true positives and false negatives
  for attribute in gold_attributes:
    all_attributes[attribute] += 1
    if attribute in predicted_attributes:
      # gold attribute exists in predicted attributes
      # some attributes have multiple values
      predicted_values = predicted_attributes[attribute].split(',')
      gold_values = gold_attributes[attribute].split(',')
      for value in gold_values:
        if value.strip() in predicted_values:
          # gold attribute exists in predicted attributes and gold value exists in predicted values
          true_positives[attribute] += 1
        else:
          # gold attribute exists in predicted attributes but gold value does not exist in predicted values
          false_negatives[attribute] += 1
    else:
      # gold attribute does not exist in predicted attributes
      false_negatives[attribute] += 1

  # Compute false positives
  for attribute in predicted_attributes:
    if attribute not in gold_attributes:
      # predicted attribute does not exist in gold attributes
      all_attributes[attribute] += 1
      false_positives[attribute] += 1
    else:
      # predicted attribute exists in gold attributes
      # some attributes have multiple values
      gold_values = gold_attributes[attribute].split(',')
      predicted_values = predicted_attributes[attribute].split(',')
      for value in predicted_values:
        if value.strip() not in gold_values:
          # predicted attribute exists in gold attributes but predicted value does not exist in gold values
          false_positives[attribute] += 1

In [476]:

def evaluate_on_test_set(finetuned_bio_parser_model, wiki_bio_test, threshold_count = 5):
  """
  Computer the precision, recall and f-score for each of the attributes
  that appears more than the treshold count
  """
  true_positives = Counter()
  false_positives = Counter()
  false_negatives = Counter()
  all_attributes = Counter()

  for bio, gold_attributes in wiki_bio_test:
    predicted_attributes = convert_to_dict(parse_bio(bio, finetuned_bio_parser_model))
    update_counts(gold_attributes, predicted_attributes, true_positives, false_positives, false_negatives, all_attributes)

  average_precision = 0
  average_recall = 0
  total = 0

  for attribute in all_attributes:
    if all_attributes[attribute] < threshold_count:
      continue
    print(attribute.upper())
    try:
      precision = true_positives[attribute] / (true_positives[attribute] + false_positives[attribute])
    except:
      precision = 0.0
    try:
      recall = true_positives[attribute] / (true_positives[attribute] + false_negatives[attribute])
    except:
      recall = 0.0
    print("precision:", precision)
    print("recall:", recall)
    print("f-score:", (precision+recall)/2)
    print('---')
    average_precision += precision
    average_recall += recall
    total += 1

  print("AVERAGE")
  average_precision = average_precision/total
  average_recall = average_recall/total
  print("precision:", average_precision)
  print("recall:", average_recall)
  print("f-score:", (average_precision+average_recall)/2)
  print('---')


If you would like to evaluate on the full test set, there are 237 test items.  You can set `max_test_items=237`.  Doing so will call your `parse_bio` function about 237 times, so be sure that you've got it properly debugged and working before running this code.

# **Important: Grading Criteria**

For a thorough evaluation of the model, max_test_items has to be set to a **minimum of 50**. To receive full credit, you have to submit a previously run notebook with all outputs included. Full credit will be awarded if **all three metrics—average precision, average recall, and average f-score—exceed 0.4 (40%) on at least 50 examples**.

In [504]:
# Delete this later
testset_filename = 'SynthBio_test.json'
max_test_items = 5  # set to 10 for initial testing; for final testing and submission, choose a value between 50 and 237
wiki_bio_test = load_wiki_bio_test_set(testset_filename, max_test_items)
# print(wiki_bio_test)
for bio, gold_attributes in wiki_bio_test:
    print(gold_attributes)
    print("gold_attributes:^^^^^^^^^^^^^^^^^^^^^^")
    print(bio)
    predicted_attributes = convert_to_dict(parse_bio(bio, finetuned_bio_parser_model))
    print(predicted_attributes)
    print("predicted attributes:^^^^^^^^^^^^^^^^^")

{'name': 'Igor Ivanov', 'gender': 'male', 'nationality': 'Russian', 'birth_date': '18 October 1970', 'birth_place': 'Moscow, Russian Soviet Federative Socialist Republic', 'known_for': 'paintings of portraits, city scenes, still lifes, and flowers', 'notable_works': 'Portraits of Celebrities, Still Life with Flowers, Flowers and Fruit', 'movement': 'realism', 'alma_mater': 'the Moscow College of Art (now in the Moscow Institute of Painting, Sculpture and Architecture)', 'awards': "People's Artist of the USSR, Gold Medals, Hero of Socialist Labour", 'elected': "People's Artist of Russia, Professor of the Academy", 'mother': 'Anastasia Nikolaevna Ivanova', 'father': 'Nikolay Ivanovich Ivanov', 'partner': 'Alexandra Alexandrovna Vasilieva', 'children': 'Alexandra Ivanovna Vasilieva, Anastasia Ivanovna Vasilieva', 'notable_type': 'artist'}
gold_attributes:^^^^^^^^^^^^^^^^^^^^^^
Igor Ivanov was born on October 18, 1970 in Moscow, Russia. His mother is Anastasia Nikolaevna Ivanova, and his f

In [525]:
testset_filename = 'SynthBio_test.json'
max_test_items = 50  # set to 10 for initial testing; for final testing and submission, choose a value between 50 and 237
wiki_bio_test = load_wiki_bio_test_set(testset_filename, max_test_items)
evaluate_on_test_set(finetuned_bio_parser_model, wiki_bio_test, threshold_count=5)

NAME
precision: 0.8461538461538461
recall: 0.22
f-score: 0.5330769230769231
---
GENDER
precision: 0.8809523809523809
recall: 0.74
f-score: 0.8104761904761905
---
NATIONALITY
precision: 0.8260869565217391
recall: 0.76
f-score: 0.7930434782608695
---
BIRTH_DATE
precision: 0.6744186046511628
recall: 0.58
f-score: 0.6272093023255814
---
BIRTH_PLACE
precision: 0.47368421052631576
recall: 0.36486486486486486
f-score: 0.4192745376955903
---
KNOWN_FOR
precision: 0.17857142857142858
recall: 0.16666666666666666
f-score: 0.17261904761904762
---
NOTABLE_WORKS
precision: 0.21428571428571427
recall: 0.2857142857142857
f-score: 0.25
---
MOVEMENT
precision: 0.3333333333333333
recall: 0.3333333333333333
f-score: 0.3333333333333333
---
ALMA_MATER
precision: 0.2
recall: 0.26666666666666666
f-score: 0.23333333333333334
---
AWARDS
precision: 0.3448275862068966
recall: 0.2777777777777778
f-score: 0.3113026819923372
---
ELECTED
precision: 0.375
recall: 0.375
f-score: 0.375
---
MOTHER
precision: 0.73913043478

How well did your model perform?

In [ ]:
# TODO - fill in these values
average_precision = 0.0
average_recall = 0.0
average_fscore = 0.0

# What attributes had the highest F-scores
best_attributes = {
    "attrbute_name1" : 0.0,
    "attrbute_name2" : 0.0,
    "attrbute_name3" : 0.0,
}

# What attributes had the lowest F-scores
worst_attributes = {
    "attrbute_name1" : 0.0,
    "attrbute_name2" : 0.0,
    "attrbute_name3" : 0.0,
}

# What could you do the perform the model's performance?
potential_improvements = """
TODO
"""

# Feedback questions

In [ ]:
# How many hours did you spend on this assignment? Just an approximation is fine.
num_hours_spent = 0

# What did you think?  This was the first time we tried this assignment
# so you're feedback is valable.
feedback = """
Type your response here.
Your response may span multiple lines.
Do not include these instructions in your response.
"""